<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/2-cross-encoder/2_Cross_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2 Assignment: Cross-Encoder

In [1]:
!pip install transformers -q
!pip install pyserini -q
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

This notebook is based on [this one](https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing#scrollTo=wHeZ9nAOEB0U) provided in the assignment description. When a cell has with "Uses the suggested notebook", it means that the code came from there.

In [35]:
import random
import torch
import torch.nn.functional as F
import numpy as np

from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader
from torch.utils import data

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup, BatchEncoding

from torch import nn
from torch import optim
from tqdm.auto import tqdm
from statistics import mean

In [5]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

## Preparing the data

In [7]:
# Download training data
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

--2023-03-15 01:28:43--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.128, 173.194.214.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  --.-KB/s    in 0.07s   

2023-03-15 01:28:44 (115 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]



According to ChatGPT:

The MSMARCO Triples Tiny dataset is a smaller subset of the full MSMARCO Triples dataset that contains 3-tuples of (query, positive document, negative document) pairs. Each triple in the MSMARCO Triples Tiny dataset consists of the following fields:

* Query: The search query that was issued by the user.
* Positive Document: A document that is relevant to the query and is considered as a positive example.
* Negative Document: A document that is irrelevant to the query and is considered as a negative example.

All three fields are represented as strings in the dataset. The MSMARCO Triples Tiny dataset is designed to be used for training and evaluating machine learning models for query-document ranking and relevance prediction tasks. The dataset is smaller than the full MSMARCO Triples dataset, containing only 31,000 triples, but can be used for quick experimentation and prototyping of machine learning models.

In [8]:
# Code from ChatGPT
df = pd.read_csv('msmarco_triples.train.tiny.tsv', 
                 delimiter = '\t', 
                 header = None, 
                 names = ['query', 'pos_doc', 'neg_doc'])

df.head()

,query,pos_doc,neg_doc
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [9]:
distinct_queries = len(pd.unique(df['query']))
total_queries = len(df['query'])

print(f'The train dataset has {distinct_queries} distinct queries of {total_queries} in total. That is: there are {total_queries - distinct_queries} repeated queries.')

The train dataset has 10779 distinct queries of 11000 in total. That is: there are 221 repeated queries.


In order to use the dataset to train a binary classifier, we should change the format of the dataset to something that has:
- query
- document
- relevant or not

Thanks, Mirelle, for the idea to use `pandas.melt` to solve this problem!



In [10]:
# Because pandas.melt unpivots da dataframe, it is useful to rename the columns
# so that the corresponding values will be correct
df.rename(columns = {'pos_doc': 'True', 'neg_doc': 'False'}, inplace = True)

train_df = pd.melt(
    df,
    id_vars = ['query'],
    value_vars = ['True', 'False'],
    var_name = 'relevance',
    value_name = 'document'
)

train_df['relevance'] = train_df['relevance'].astype('bool')

In [11]:
train_df.head()

,query,relevance,document
0,is a little caffeine ok during pregnancy,True,We donât know a lot about the effects of caf...
1,what fruit is native to australia,True,Passiflora herbertiana. A rare passion fruit n...
2,how large is the canadian military,True,The Canadian Armed Forces. 1 The first large-...
3,types of fruit trees,True,Cherry. Cherry trees are found throughout the ...
4,how many calories a day are lost breastfeeding,True,"Not only is breastfeeding better for the baby,..."


In [12]:
train_df.shape

(22000, 3)

In [37]:
# Saving the new df for posterior use
train_df.to_csv('format_msmarco_triples.train.tiny.tsv',
                sep = '\t',
                index = False)

I don't really see any problem with "leak" between train and test datasets. Here, I separate 5% for testing, 15% for validation and 80% for training.

In [39]:
df_train = train_df.sample(frac = 0.8, random_state = 42)                       # 80%
df_valid = train_df.drop(df_train.index).sample(frac = 0.75, random_state = 42) # 15%
df_test = train_df.drop(df_train.index).drop(df_valid.index)                    # 5%

In [41]:
df_train.shape, df_valid.shape, df_test.shape

((17600, 3), (3300, 3), (1100, 3))

In [42]:
df_train.head()

,query,relevance,document
13035,where is fitbit connect,True,Charging your tracker ...........................
3115,defense messaging system,True,Defense Message System. The Defense Message Sy...
8732,What are the four main ocean basins? which is ...,True,Although both the Northern and Southern Hemisp...
7591,"how many hours a day to work to get lunch in ,a",True,"In the United States, most states require a mi..."
221,can pizza boxes be recycled,True,Pizza box suitable for recycling. Pizza box no...


In [43]:
df_valid.head()

,query,relevance,document
1367,can energy drinks clean your system,True,You cannot clean your body with energy drinks....
17396,is ilovemakonnen ovo,True,"If you're a pescatarian, you can eat any type ..."
21689,process of inserting gene into bacterium,True,Alternative Splicing. Alternative splicing is ...
5136,what are you if you born in june,True,June 5 Career. The combination of charm and in...
8007,what binds with uracil,True,Uracil /ËjÊÉrÉsÉªl/ (U) is one of the four...


In [44]:
df_test.head()

,query,relevance,document
16,numbers to letters in the alphabet,True,The modern English alphabet is a Latin alphabe...
24,what is the formula of aluminum carbide,True,"Aluminium carbide, chemical formula Al 4 C 3, ..."
25,how many hours can someone work before you hav...,True,The PPACAâs requirement for employers to off...
60,wow release date,True,WoW: Legion release date. Legion's release dat...
77,what are good food sources for magnesium,True,Magnesium Rich Food Sources. Green vegetables ...


In [45]:
# y_train = df_train.pop('relevance')
# X_train = df_train

# y_test = df_test.pop('relevance')
# X_test = df_test

In [46]:
# X_train.head()

## BM25 base search
For the index, we'll use one of [Pyserini's Prebuilt indexes](https://github.com/castorini/pyserini/blob/master/docs/prebuilt-indexes.md). Following the steps from the documentation:

In [47]:
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage-slim')
searcher.set_bm25(k1 = 0.9, b = 0.4)

In [48]:
index_reader = IndexReader.from_prebuilt_index('msmarco-v1-passage-slim')
index_reader.stats()

{'total_terms': 352316036,
 'documents': 8841823,
 'non_empty_documents': 8841823,
 'unique_terms': 2660824}

In [49]:
max_length = 356

In [50]:
# Mirelle had the most amazing way of putting this together, so I thank her for the inspiration and code
# It uses a lot of the suggested notebook
class MSMARCODataset(data.Dataset):
    def __init__(self, tokenizer, query, documents, targets, max_lenght = 356):
        self.tokenizer = tokenizer
        self.query = query
        self.documents = documents
        self.targets = targets
        self.max_lenght = max_lenght
    
    def __len__(self):
        return len(self.query)

    def __getitem__(self, idx):
        # Using prompt to concatenate query + passage
        instruction = 'Query: ' + self.query[idx]+ '  Returns: '+self.documents[idx]+' ?'
        instruction_token = self.tokenizer(instruction,
                                           max_length = self.max_lenght, 
                                           truncation = True,
                                           padding = "max_length", 
                                           return_tensors = 'pt')


        return {'input_ids': torch.squeeze(instruction_token['input_ids']).long().to(device), \
               'attention_mask': torch.squeeze(instruction_token['attention_mask']).long().to(device), \
               'labels': int(self.targets[idx])}

In [51]:
model_name = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [53]:
device = (
    torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)
print(device)

cuda


In [54]:
# This cell uses the suggested notebook
# This functions adds "pad" tokens to examples in the batch that are shorter than the largest one.
def collate_fn(batch):
    return BatchEncoding(tokenizer.pad(batch, return_tensors = 'pt'))

# Convert examples to Pytorch's Dataset.
train_dataset = MSMARCODataset(
    tokenizer, 
    df_train['query'].to_list(), 
    df_train['document'].to_list(), 
    df_train['relevance'].to_list()
)

valid_dataset = MSMARCODataset(
    tokenizer, 
    df_valid['query'].to_list(), 
    df_valid['document'].to_list(), 
    df_valid['relevance'].to_list()
)


test_dataset = MSMARCODataset(
    tokenizer, 
    df_test['query'].to_list(), 
    df_test['document'].to_list(), 
    df_test['relevance'].to_list()
)

# Convert examples to Pytorch's DataLoader.
dataloader_train = data.DataLoader(train_dataset, batch_size = 32, shuffle = True, collate_fn = collate_fn)
dataloader_valid = data.DataLoader(valid_dataset, batch_size = 32, shuffle = True, collate_fn = collate_fn)
dataloader_test = data.DataLoader(test_dataset, batch_size = 32, shuffle = False, collate_fn = collate_fn)

In [55]:
# Uses the suggested notebook
for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Finetuning

In [56]:
# Uses the suggested notebook
# We first define the evaluation function to measure accuracy and loss

def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
print('Parameters', model.num_parameters())

epochs = 5
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(dataloader_train)
# Warm up is important to stabilize training.
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# First validation to check if evaluation code is working and accuracy is random as expected 
evaluate(model = model, dataloader = dataloader_valid, set_name='Valid')

# Training loop
for epoch in tqdm(range(epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
        optimizer.zero_grad()
        outputs = model(**batch.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.cpu().item())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valis:   0%|          | 0/104 [00:00<?, ?it/s]

Valis loss: 0.708; Valis accuracy: 0.000


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Train:   0%|          | 0/550 [00:00<?, ?it/s]

Train:   0%|          | 0/550 [00:00<?, ?it/s]

## Reranking
